## Занятие 4. Разработка ml проектов: хорошие и плохие практики
На занятии 3 мы написали пайплайн обучения семантического сегментатора подводных фото в одном jupyter ноутбуке.

На этом занятии мы оформим этот пайплайн в ml проект так, чтобы его можно было поддерживать и развивать.


In [1]:
from pathlib import Path
ROOT_PATH = Path().absolute()
assert ROOT_PATH.name == 'c04', ROOT_PATH.name
ROOT_PATH

WindowsPath('D:/edu/course_cvdl/classes/c04')

### Плохая практика 1: использование jupyter ноутбуков в качестве основного средства разработки
- https://www.kdnuggets.com/2019/11/notebook-anti-pattern.html
- https://analyticsindiamag.com/an-argument-against-using-jupyter-notebook-for-machine-learning/
- https://medium.com/skyline-ai/jupyter-notebook-is-the-cancer-of-ml-engineering-70b98685ee71

Главные проблемы:
- нелинейность исполнения кода (сложно читать и сложно воспроизводить)
- невозможность тестирования
- нечитыемые diff-ы в git (сложно разобраться в истории и работать совместно)

Jupyter ноутбуки хорошо подходят чтобы:
- визуализировать данные
- набросать прототип
- поделиться однократным результатом


#### **Решение**: 
Использовать стандартные инструменты разработки ЯП, например - Python пакеты.

In [2]:
# Python ищет пакеты в PYTHONPATH, если пакет не найден - то получаем ModuleNotFound
from suim_segmentation.data import SuimDataset

ModuleNotFoundError: No module named 'suim_segmentation'

In [3]:
cd {ROOT_PATH}/src

D:\edu\course_cvdl\classes\c04\src


In [4]:
# В PYTHONPATH всегда содержится current_dir, так что из родительской директории можно импортировать пакет
from suim_segmentation.data import SuimDataset

In [5]:
from suim_segmentation import data as suim_data
suim_data.__file__

'D:\\edu\\course_cvdl\\classes\\c04\\src\\suim_segmentation\\data.py'

Если пакет находится в текущей папке, то его можно импортировать.

Это не очень удобно - код можно будет вызывать только при определенной текущей директории.

**Правильное решение - сделать пакет устанавливаемым!**

### 1.1 Пакетируем код

На сегодня (2023 год) [рекомендуется](https://packaging.python.org/en/latest/tutorials/packaging-projects/#creating-pyproject-toml) пакетировать код с помощью `pyproject.toml` файла:
- поддерживает разные системы сборки (не только setuptools)
- поддерживает C++ расширения
- стандартизирован в PEP-518, PEP-621

Раньше часто использовались другие [способы](https://packaging.python.org/en/latest/glossary/?highlight=setup.py#term-setup.py), но сейчас `pyproject.toml` является рекомендуемым

In [6]:
ls .

 Volume in drive D is Ext
 Volume Serial Number is 5670-7B72

 Directory of D:\edu\course_cvdl\classes\c04\src

01.10.2023  17:13    <DIR>          .
01.10.2023  17:13    <DIR>          ..
01.10.2023  17:13    <DIR>          suim_segmentation
               0 File(s)              0 bytes
               3 Dir(s)  351я745я527я808 bytes free


In [7]:
%%writefile {str(ROOT_PATH / 'src' / 'pyproject.toml')}

[project]
name = "suim_segmentation"
description = "ML project example package"
version = "0.1.0"

Writing D:\edu\course_cvdl\classes\c04\src\pyproject.toml


### Готово - пакет стал устанавливаемым


In [9]:
ls

 Volume in drive D is Ext
 Volume Serial Number is 5670-7B72

 Directory of D:\edu\course_cvdl\classes\c04\src

01.10.2023  17:18    <DIR>          .
01.10.2023  17:18    <DIR>          ..
01.10.2023  17:18    <DIR>          .ipynb_checkpoints
01.10.2023  17:18               104 pyproject.toml
01.10.2023  17:13    <DIR>          suim_segmentation
               1 File(s)            104 bytes
               4 Dir(s)  351я745я527я808 bytes free


In [10]:
! pip install -e .

Obtaining file:///D:/edu/course_cvdl/classes/c04/src
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for suim-segmentation (pyproject.toml): started
  Building editable for suim-segmentation (pyproject.toml): finished with status 'done'
  Created wheel for suim-segmentation: filename=suim_segmentation-0.1.0-0.editable-py3-none-any.whl size=2665 sha256=29b3bd15c320609e4e13fa659ae1830b72b0801cdccf6d5fb73d0c72a0b26a84
  Stored in directory: C:\Users\bzimka\AppData\Local\Temp\pip-ephem-wheel-cache-bluz1ha3\wheels\48\5e\2b\574c8018

Ещё пакеты c `pyproject.toml` (или `setup.py`) можно устанавливать из git репозитория, например:
`pip install git+https://github.com/arogozhnikov/einops`

In [11]:
cd {ROOT_PATH}

D:\edu\course_cvdl\classes\c04


## Restart KERNEL

In [1]:
from pathlib import Path
ROOT_PATH = Path().absolute()
assert ROOT_PATH.name == 'c04', ROOT_PATH.name
ROOT_PATH

WindowsPath('D:/edu/course_cvdl/classes/c04')

In [2]:
from suim_segmentation.data import SuimDataset
from suim_segmentation import data
data.__file__

'D:\\edu\\course_cvdl\\classes\\c04\\src\\suim_segmentation\\data.py'

In [3]:
ls

 Volume in drive D is Ext
 Volume Serial Number is 5670-7B72

 Directory of D:\edu\course_cvdl\classes\c04

01.10.2023  17:21    <DIR>          .
01.10.2023  17:21    <DIR>          ..
01.10.2023  12:29                27 .gitignore
30.09.2023  21:16    <DIR>          .ipynb_checkpoints
01.10.2023  17:21            88я955 c04.ipynb
01.10.2023  17:04    <DIR>          data
10.09.2023  10:25           645я578 dvc_scheme.png
10.09.2023  10:25               309 README.md
01.10.2023  17:20    <DIR>          src
               4 File(s)        734я869 bytes
               5 Dir(s)  351я745я511я424 bytes free


In [4]:
cd {ROOT_PATH/"src"}

D:\edu\course_cvdl\classes\c04\src


### 1.1 Проверяем код на PEP-8 с помощью `pylint`
Pylint - инструмент для проверки кода на соответствие PEP-8.

`pip install pylint`

Вывод всех нарушений PEP8 и оценки вашего кода: `python -m pylint suim_segmentation`

Вывод только ошибок: `python -m pylint suim_segmentation/ -E`

In [ ]:
! python -m pylint suim_segmentation

### 1.2 Форматируем код с помощью [black](https://github.com/psf/black) и [isort](https://github.com/PyCQA/isort)
Часть ошибок форматирования можно поправить автоматически с помощью black и isort.

`pip install black isort`

Форматирование: 
- `python -m black suim_segmentation/`
- `python -m isort suim_segmentation/`

In [15]:
! python -m black src/suim_segmentation

reformatted D:\edu\course_cvdl\classes\c04\src\suim_segmentation\metrics.py

All done! \u2728 \U0001f370 \u2728
1 file reformatted, 5 files left unchanged.


In [13]:
! python -m isort src/suim_segmentation

Fixing D:\edu\course_cvdl\classes\c04\src\suim_segmentation\data.py
Fixing D:\edu\course_cvdl\classes\c04\src\suim_segmentation\model.py
Fixing D:\edu\course_cvdl\classes\c04\src\suim_segmentation\trainer.py
Fixing D:\edu\course_cvdl\classes\c04\src\suim_segmentation\.ipynb_checkpoints\data-checkpoint.py
Fixing D:\edu\course_cvdl\classes\c04\src\suim_segmentation\.ipynb_checkpoints\model-checkpoint.py


In [ ]:
! python -m pylint suim_segmentation

### 1.3 Тюним правил PEP-8 под свой проект
Для pylint можно создать набор правил и указать его в `pyproject.toml`

In [15]:
assert ROOT_PATH.exists()

In [19]:
%%writefile {str(ROOT_PATH / 'src' / 'pyproject.toml')}

[project]
name = "suim_segmentation"
description = "ML project example package"
version = "0.1.0"

[tool.pylint]
good-names = "b,h,w,x,tp,fp,fn"


Overwriting D:\edu\course_cvdl\classes\c04\src\pyproject.toml


In [20]:
cd {ROOT_PATH/"src"}

D:\edu\course_cvdl\classes\c04\src


In [ ]:
! python -m pylint suim_segmentation

### 1.4 Пишем "точку входа" в пайплайн (скрипт обучения) 
Собираем код с прошлого занятия для запуска пайплайна

In [22]:
%%writefile {str(ROOT_PATH / 'src' / 'suim_segmentation' / 'run.py')}

import argparse
from pathlib import Path

import torch
from torch.utils import data as tdata
from tqdm import tqdm

from .data import SuimDataset, EveryNthFilterSampler
from .model import SuimModel
from .loss import DiceLoss
from .metrics import Accuracy
from .trainer import Trainer

PROJECT_NAME = 'suim_segmentation2023'


def run_pipeline(args):
    device = torch.device(args.device)
    model = SuimModel().to(device)
    model.encoder.requires_grad_(False)

    opt = torch.optim.Adam(model.parameters(), lr=args.lr)

    train_val_ds = SuimDataset(
        Path(args.train_data), masks_as_color=False, target_size=(256, 256)
    )
    test_ds = SuimDataset(
        Path(args.test_data), masks_as_color=False, target_size=(256, 256)
    )
    test_iter = tdata.DataLoader(test_ds, batch_size=args.batch_size, shuffle=False)
    train_iter = tdata.DataLoader(
        train_val_ds,
        batch_size=args.batch_size,
        sampler=EveryNthFilterSampler(
            dataset_size=len(train_val_ds), n=5, pass_every_nth=False, shuffle=True
        ),
    )
    val_iter = tdata.DataLoader(
        train_val_ds,
        batch_size=args.batch_size,
        sampler=EveryNthFilterSampler(
            dataset_size=len(train_val_ds), n=5, pass_every_nth=True, shuffle=False
        ),
    )
    loss = DiceLoss()
    metric = Accuracy()

    trainer = Trainer(
        net=model,
        opt=opt,
        train_loader=train_iter,
        val_loader=val_iter,
        test_loader=test_iter,
        loss=loss,
        metric=metric,
    )
    mean = lambda x: sum(x) / len(x)

    for e in range(args.num_epochs):
        print(f"Epoch {e}")
        with_testing = (e == args.num_epochs - 1)
        epoch_stats = trainer(num_epochs=1, with_testing=with_testing)
        train_loss, train_metric = epoch_stats['train'][0]
        val_loss, val_metric = epoch_stats['val'][0]
        assert isinstance(train_loss, list), type(train_loss)

    test_loss, test_metric = epoch_stats['test'][0]


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--name", type=str, required=True)
    parser.add_argument("--train-data", type=str, required=True)
    parser.add_argument("--test-data", type=str, required=True)
    parser.add_argument("--lr", type=float, required=True)
    parser.add_argument("--num-epochs", type=int, required=True)
    parser.add_argument("--batch-size", type=int, default=16)
    parser.add_argument("--device", type=str, default='cpu:0')
    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()
    run_pipeline(args)
    print("Finished")

Writing D:\edu\course_cvdl\classes\c04\src\suim_segmentation\run.py


In [ ]:
! python -m suim_segmentation.run --name=baseline --lr=0.03 --num-epochs=1 --batch-size=32 --device=cpu:0 \
    --train-data={ROOT_PATH}\data\train_val \
    --test-data={ROOT_PATH}\data\TEST 

### Плохая практика 2: "ручное" копирование данных

- В реальных проектах (индустрия/соревнования) часто используется несколько датасетов, а не один
- Они могут быть в разных форматах, а код проекта обычно работает только с одним форматом
- Датасет может обновляться - даже у ImageNet есть [version2](https://proceedings.mlr.press/v97/recht19a/recht19a.pdf)
- Часто датасет нужен на разных машинах (например, локальной и devbox с GPU)

#### **Решение**: 
Использовать инструменты для управления данными, например - [dvc](https://dvc.org)

![dvc scheme was not loaded](dvc_scheme.png "DVC scheme")

### 2.1 Добавляем google-drive в качестве хранилища

https://dvc.org/doc/user-guide/how-to/setup-google-drive-remote

In [24]:
cd {ROOT_PATH}/data

D:\edu\course_cvdl\classes\c04\data


In [25]:
! dvc init --subdir

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [26]:
! dvc remote add mygoogledrive gdrive://1_8FVmJgPW-dwYr8jOe9PQupCEy53WQ4d

In [27]:
! dvc remote list

mygoogledrive	gdrive://1_8FVmJgPW-dwYr8jOe9PQupCEy53WQ4d


Добавим test-данные в индекс dvc

In [28]:
ls

 Volume in drive D is Ext
 Volume Serial Number is 5670-7B72

 Directory of D:\edu\course_cvdl\classes\c04\data

01.10.2023  17:42    <DIR>          .
01.10.2023  17:42    <DIR>          ..
01.10.2023  17:42    <DIR>          .dvc
01.10.2023  17:42               142 .dvcignore
30.09.2023  22:18                26 .gitignore
30.09.2023  18:28    <DIR>          test
30.09.2023  19:23    <DIR>          train_val
               2 File(s)            168 bytes
               5 Dir(s)  351я603я404я800 bytes free


In [29]:
! dvc add test


To track the changes with git, run:

	git add test.dvc

To enable auto staging, run:

	dvc config core.autostage true


\u280b Checking graph



Отправим данные в хранилище

In [31]:
! dvc push test.dvc --remote=mygoogledrive

551 files pushed


Тперь можно сохранить "ссылки" на данные в git.
```
! git add data/
! git commit -m "Add data with DVC"
```

In [32]:
# скачаем данные test
! dvc pull test.dvc

A       test\
1 file added


### Плохая практика 3: проведение экспериментов без отслеживания результатов
Достижение лучшх результатов в любой ml-задаче требует множество экспериментов, каждый из которых дает небольшое улучшение (или ухудшение).

Результаты экспериментов необходимо сравнивать между собой, чтобы оставлять успешные идеи и отбрасывать неудачные.

Простейший (ручной) способ отслеживания экспериментов:
- выполнили эксперимент N, запомнили метрики
- выполнили эксперимент N+1, сравнили метрики с N, запомнили
- выполнили эксперимент N+2, сравнили метрики с N+1, запомнили
- ...

Главная проблема: **после эксперимента не остаётся следов (артефактов)**

Проблемы-следствия:
- нельзя сравнить результаты эксперименты N и (N+10) 
- сложно провести многовариантный, а не бинарный эксперимент
- сложно воспроизвести идею N (если она стала снова актуальной)
- нужно держать в голове, насколько хорощо/плохо сработала идея когда-то в прошлом

**Решение:** Логировать все параметры и результаты эксперимента

### 3.1 Подключаем Weights & Biases
https://docs.wandb.ai/quickstart

1. Устанавливаем wandb: `! pip install wandb`
2. Авторизуемся на `https://wandb.ai/login` (например, через GitHub)
3. Заходим на `https://wandb.ai/settings`, копируем ключ из `API keys`
4. Выполняем `$ wandb login <YOUR API KEY>` 

### 3.2 Проверяем wnb

In [33]:
import wandb

In [34]:
wandb.init(project='c04', config={'lr': 0.01, 'foo': 'bar', 'something': True}, name='first')

wandb: Currently logged in as: zimka. Use `wandb login --relogin` to force relogin


Можно логировать численные величины. Каждое вызов log неявно увеличивает внутренний счётчик шагов.

In [35]:
wandb.log({"train": {"loss": 0.9, "metric": 0.5}, "val": {"loss": 0.4, "acc": 0.8}})

In [36]:
wandb.log({"train": {"loss": 0.8, "metric": 0.5}, "val": {"loss": 0.35, "acc": 0.8}})

In [37]:
wandb.log({"train": {"loss": 0.75, "metric": 0.52}, "val": {"loss": 0.33, "acc": 0.7}})

In [38]:
wandb.log({"train": {"loss": 0.74, "metric": 0.52}, "val": {"loss": 0.32, "acc": 0.7}})

Можно явно указать шаг, к которому относится запись

In [42]:
wandb.log({"train": {"loss": 0.74, "metric": 0.52}, "val": {"loss": 0.32, "acc": 0.7}}, step=20)

Каждый вызов log добавляет аргументы во внутреннее состояние и коммитит **предыдущие** значения.

Можно считать, что каждый вызов .log - это `git commit` старых данных + `git add` новых данных.

In [40]:
wandb.log({"train": {"loss": 0.74, "metric": 0.52}, "val": {"loss": 0.32, "acc": 0.7}}, step=50)

Можно логировать не только численные величины - например, изображения с масками.

In [43]:
from suim_segmentation.data import SuimDataset

test_data = SuimDataset(root=ROOT_PATH / 'data' / 'TEST', masks_as_color=False)

110it [00:01, 79.86it/s]


In [44]:
x_img, y_mask = test_data[2]

In [45]:
SuimDataset.LABEL_COLORS

(('Background(waterbody)', '000'),
 ('Human divers', '001'),
 ('Aquatic plants and sea-grass', '010'),
 ('Wrecks and ruins', '011'),
 ('Robots (AUVs/ROVs/instruments)', '100'),
 ('Reefs and invertebrates', '101'),
 ('Fish and vertebrates', '110'),
 ('Sea-floor and rocks', '111'))

In [46]:
class_labels = dict(
    (num, cls_name) for num, (cls_name, binary_idx) in enumerate(SuimDataset.LABEL_COLORS)
)

mask_img = wandb.Image(x_img.permute(1, 2, 0).numpy(), masks={
  "predictions": {
    "mask_data": y_mask[0].numpy(),
    "class_labels": class_labels
  }
})

In [47]:
wandb.log({"gt_example": mask_img}, commit=True)

Можно добавить ключ-значение в summary

In [48]:
wandb.run.summary['my_key'] = 'my_important_value'

In [49]:
wandb.finish()

my_key,my_important_value


### Залогируем параметры и результаты эксперимента
Дописать код run.py так, чтобы логировались (как минимум):
- Средние train.loss, train.metric, val.loss, val.metric для каждой эпохи
- Средние test.loss, test.metric однократно


In [50]:
%%writefile {str(ROOT_PATH / 'src' / 'suim_segmentation' / 'run.py')}
import wandb
import argparse
from pathlib import Path

import torch
from torch.utils import data as tdata
from tqdm import tqdm

from .data import SuimDataset, EveryNthFilterSampler
from .model import SuimModel
from .loss import DiceLoss
from .metrics import Accuracy
from .trainer import Trainer

PROJECT_NAME = 'suim_segmentation2023'


def run_pipeline(args):
    device = torch.device(args.device)
    model = SuimModel().to(device)
    model.encoder.requires_grad_(False)

    opt = torch.optim.Adam(model.parameters(), lr=args.lr)

    train_val_ds = SuimDataset(
        Path(args.train_data), masks_as_color=False, target_size=(256, 256)
    )
    test_ds = SuimDataset(
        Path(args.test_data), masks_as_color=False, target_size=(256, 256)
    )
    test_iter = tdata.DataLoader(test_ds, batch_size=args.batch_size, shuffle=False)
    train_iter = tdata.DataLoader(
        train_val_ds,
        batch_size=args.batch_size,
        sampler=EveryNthFilterSampler(
            dataset_size=len(train_val_ds), n=5, pass_every_nth=False, shuffle=True
        ),
    )
    val_iter = tdata.DataLoader(
        train_val_ds,
        batch_size=args.batch_size,
        sampler=EveryNthFilterSampler(
            dataset_size=len(train_val_ds), n=5, pass_every_nth=True, shuffle=False
        ),
    )
    loss = DiceLoss()
    metric = Accuracy()

    trainer = Trainer(
        net=model,
        opt=opt,
        train_loader=train_iter,
        val_loader=val_iter,
        test_loader=test_iter,
        loss=loss,
        metric=metric,
    )
    mean = lambda x: sum(x) / len(x)

    wandb.init(
        project=PROJECT_NAME, name=args.name,
        config=vars(args)
    )
    for e in range(args.num_epochs):
        print(f"Epoch {e}")
        with_testing = (e == args.num_epochs - 1)
        epoch_stats = trainer(num_epochs=1, with_testing=with_testing)
        train_loss, train_metric = epoch_stats['train'][0]
        val_loss, val_metric = epoch_stats['val'][0]
        assert isinstance(train_loss, list), type(train_loss)
        wandb.log({
            "train": {"loss": mean(train_loss), "metric": mean(train_metric)},
            "val": {"loss": mean(val_loss), "metric": mean(val_metric)}
        })
    
    test_loss, test_metric = epoch_stats['test'][0]
    wandb.summary['test.loss'] = mean(test_loss)
    wandb.summary['test.metric'] = mean(test_metric)
    wandb.finish()


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--name", type=str, required=True)
    parser.add_argument("--train-data", type=str, required=True)
    parser.add_argument("--test-data", type=str, required=True)
    parser.add_argument("--lr", type=float, required=True)
    parser.add_argument("--num-epochs", type=int, required=True)
    parser.add_argument("--batch-size", type=int, default=16)
    parser.add_argument("--device", type=str, default='cpu:0')
    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()
    run_pipeline(args)
    print("Finished")

Overwriting D:\edu\course_cvdl\classes\c04\src\suim_segmentation\run.py


In [ ]:
! python -m suim_segmentation.run --name=baseline --lr=0.03 --num-epochs=1 --batch-size=32 --device=cpu:0 \
    --train-data=D:\edu\course_cvdl\classes\c04\data\train_val \
    --test-data=D:\edu\course_cvdl\classes\c04\data\TEST 

## Запуск в DataSphere
```
%pip install wandb
!python3 -m wandb login <your key>
```

In [54]:
#!g2.mig
from pathlib import Path
ROOT_PATH = Path().absolute()
assert ROOT_PATH.name == 'c04', ROOT_PATH.name
ROOT_PATH

WindowsPath('D:/edu/course_cvdl/classes/c04')

## 4. Добавим в пайплайн аугментаций `albumentations`
https://albumentations.ai/docs/examples/pytorch_semantic_segmentation/

In [ ]:
#!g2.mig
from suim_segmentation.data import SuimDataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

AUG = A.Compose([
    A.OneOf([
        A.RandomSizedCrop(min_max_height=(150, 250), height=256, width=256, p=0.5),
        A.HorizontalFlip(p=0.5),
    ],p=1),
    A.OneOf([
        A.ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        A.GridDistortion(p=0.5),
        A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1),
    ], p=0.8),
    ToTensorV2()
])


class SuimDatasetWAug(SuimDataset):
    def __init__(self, *, transform=None, **kwargs):
        assert kwargs.get('masks_as_colors', False) == False, "Not supported"
        super().__init__(**kwargs)
        self.transform = transform
    
    def __getitem__(self, idx: int):
        img, mask = super().__getitem__(idx)
        if self.transform is not None:
            results = self.transform(image=img.permute(1, 2, 0).numpy(), mask=mask.numpy()[0])
            img = results['image']
            mask = results['mask'][None]
        return img, mask

ds = SuimDatasetWAug(
    root=Path('/home/jupyter/mnt/datasets/SUIM_Dataset/TEST'),
    masks_as_color=False, 
    target_size=(256, 256),
    transform=AUG
)

In [ ]:
#!g2.mig
from PIL import Image
from matplotlib import pyplot as plt
from torchvision import transforms

to_img = transforms.ToPILImage()
x, y = ds[2]
plt.imshow(to_img(x))

In [ ]:
#!g2.mig
%%writefile {str(ROOT_PATH / 'src' / 'suim_segmentation' / 'run.py')}
import wandb
import argparse
from pathlib import Path

import torch
from torch.utils import data as tdata
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from .data import SuimDataset, EveryNthFilterSampler
from .model import SuimModel
from .loss import DiceLoss
from .metrics import Accuracy
from .trainer import Trainer

PROJECT_NAME = 'suim_segmentation2023'

AUG = A.Compose([
    A.OneOf([
        A.RandomSizedCrop(min_max_height=(150, 250), height=256, width=256, p=0.5),
        A.HorizontalFlip(p=0.5),
    ],p=1),
    A.OneOf([
        A.ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        A.GridDistortion(p=0.5),
        A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1),
    ], p=0.8),
    ToTensorV2()
])


class SuimDatasetWAug(SuimDataset):
    def __init__(self, *, transform=None, **kwargs):
        assert kwargs.get('masks_as_colors', False) == False, "Not supported"
        super().__init__(**kwargs)
        self.transform = transform
    
    def __getitem__(self, idx: int):
        img, mask = super().__getitem__(idx)
        if self.transform is not None:
            results = self.transform(image=img.permute(1, 2, 0).numpy(), mask=mask.numpy()[0])
            img = results['image']
            mask = results['mask'][None]
        return img, mask


def run_pipeline(args):
    device = torch.device(args.device)
    model = SuimModel().to(device)
    model.encoder.requires_grad_(False)
    
    opt = torch.optim.Adam(model.parameters(), lr=args.lr)
    
    if args.with_augs:
        train_val_ds = SuimDatasetWAug(
            root=Path(args.train_data), masks_as_color=False, target_size=(256, 256),
            transform=AUG
        )
    else:
        train_val_ds = SuimDataset(
            root=Path(args.train_data), masks_as_color=False, target_size=(256, 256),
        )
    test_ds = SuimDataset(
        root=Path(args.test_data), masks_as_color=False, target_size=(256, 256)
    )
    test_iter = tdata.DataLoader(test_ds, batch_size=args.batch_size, shuffle=False)
    train_iter = tdata.DataLoader(
        train_val_ds,
        batch_size=args.batch_size,
        sampler=EveryNthFilterSampler(
            dataset_size=len(train_val_ds), n=5, pass_every_nth=False, shuffle=True
        ),
    )
    val_iter = tdata.DataLoader(
        train_val_ds,
        batch_size=args.batch_size,
        sampler=EveryNthFilterSampler(
            dataset_size=len(train_val_ds), n=5, pass_every_nth=True, shuffle=False
        ),
    )
    loss = DiceLoss()
    metric = Accuracy()

    trainer = Trainer(
        net=model,
        opt=opt,
        train_loader=train_iter,
        val_loader=val_iter,
        test_loader=test_iter,
        loss=loss,
        metric=metric,
    )
    mean = lambda x: sum(x) / len(x)

    wandb.init(
        project=PROJECT_NAME, name=args.name,
        config=vars(args)
    )
    for e in range(args.num_epochs):
        print(f"Epoch {e}")
        with_testing = (e == args.num_epochs - 1)
        epoch_stats = trainer(num_epochs=1, with_testing=with_testing)
        train_loss, train_metric = epoch_stats['train'][0]
        val_loss, val_metric = epoch_stats['val'][0]
        assert isinstance(train_loss, list), type(train_loss)
        wandb.log({
            "train": {"loss": mean(train_loss), "metric": mean(train_metric)},
            "val": {"loss": mean(val_loss), "metric": mean(val_metric)}
        })

    test_loss, test_metric = epoch_stats['test'][0]
    wandb.summary['test.loss'] = mean(test_loss)
    wandb.summary['test.metric'] = mean(test_metric)
    wandb.finish()


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--name", type=str, required=True)
    parser.add_argument("--train-data", type=str, required=True)
    parser.add_argument("--test-data", type=str, required=True)
    parser.add_argument("--lr", type=float, required=True)
    parser.add_argument("--num-epochs", type=int, required=True)
    parser.add_argument("--batch-size", type=int, default=16)
    parser.add_argument("--device", type=str, default='cpu:0')
    parser.add_argument("--with-augs", action='store_true')
    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()
    run_pipeline(args)
    print("Finished")

In [ ]:
#!g2.mig
! cd src & python3 -m suim_segmentation.run --name=baseline-with_augs --lr=0.03 --num-epochs=20 --batch-size=32 --device=cuda:0 \
    --train-data=/home/jupyter/mnt/datasets/SUIM_Dataset/train_val \
    --test-data=/home/jupyter/mnt/datasets/SUIM_Dataset/TEST --with-augs